In [2]:
import polars as pl

from src.config import DATA_PATH

In [7]:
canada_sales = pl.read_csv(DATA_PATH / "canada_sales.csv", infer_schema_length=10000)
usa_sales = pl.read_csv(DATA_PATH / "usa_sales.csv", infer_schema_length=10000)

In [8]:
canada_sales

Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
str,str,str,str,str,str
"""176558""","""USB-C Charging Cable""","""2""","""11.95""","""04/19/19 08:46""","""128 Elm St, Quebec City"""
null,null,null,null,null,"""113 Oak St, Ottawa"""
"""176559""","""Bose SoundSport Headphones""","""1""","""99.99""","""04/07/19 22:30""","""116 Elm St, Winnipeg"""
"""176560""","""Google Phone""","""1""","""600""","""04/12/19 14:38""","""149 Broadway, Edmonton"""
"""176560""","""Wired Headphones""","""1""","""11.99""","""04/12/19 14:38""","""121 Maple St, Vancouver"""
…,…,…,…,…,…
"""194090""","""Google Phone""","""1""","""600""","""04/08/19 17:11""","""124 Market St, Quebec City"""
"""194091""","""AA Batteries (4-pack)""","""1""","""3.84""","""04/15/19 16:02""","""130 Cedar Ave, Ottawa"""
"""194092""","""AAA Batteries (4-pack)""","""2""","""2.99""","""04/28/19 14:36""","""137 Maple St, Vancouver"""


In [9]:
usa_sales

Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
str,str,str,str,str,str
"""236670""","""Wired Headphones""","""2""","""11.99""","""08/31/19 22:21""","""359 Spruce St, Seattle, WA 981…"
"""236671""","""Bose SoundSport Headphones""","""1""","""99.99""","""08/15/19 15:11""","""492 Ridge St, Dallas, TX 75001"""
"""236672""","""iPhone""","""1""","""700.0""","""08/06/19 14:40""","""149 7th St, Portland, OR 97035"""
"""236673""","""AA Batteries (4-pack)""","""2""","""3.84""","""08/29/19 20:59""","""631 2nd St, Los Angeles, CA 90…"
"""236674""","""AA Batteries (4-pack)""","""2""","""3.84""","""08/15/19 19:53""","""736 14th St, New York City, NY…"
…,…,…,…,…,…
"""248146""","""Bose SoundSport Headphones""","""1""","""99.99""","""08/29/19 22:19""","""868 Hickory St, San Francisco,…"
"""248147""","""AAA Batteries (4-pack)""","""3""","""2.99""","""08/31/19 16:26""","""206 Lakeview St, Boston, MA 02…"
"""248148""","""AA Batteries (4-pack)""","""1""","""3.84""","""08/02/19 07:25""","""568 13th St, Seattle, WA 98101"""
